# Run Fugue + Nixtla

## Import and Modulize

Accoring to the Ray experiment, we created two steps, generating data and forecasting.

In [ ]:
import argparse
import os
from time import time

import pandas as pd
from statsforecast.utils import generate_series
from statsforecast.models import auto_arima
from statsforecast.core import StatsForecast

In [ ]:
def generate_data(n):
    df = generate_series(n_series=n, seed=1).reset_index()
    return df.assign(unique_id=df.unique_id.astype(int))

# schema: *-y+auto_arima:double
def forecast(df:pd.DataFrame) -> pd.DataFrame:
    tdf = df.set_index("unique_id")
    model = StatsForecast(tdf,models=[auto_arima], freq='D',n_jobs=1)
    return model.forecast(7).reset_index()

Now test them:

In [ ]:
forecast(generate_data(1))

Out[3]:

,unique_id,ds,auto_arima
0,0,2000-03-28,1.922440
1,0,2000-03-29,1.514246
2,0,2000-03-30,1.192725
3,0,2000-03-31,0.939472
4,0,2000-04-01,0.739993
5,0,2000-04-02,0.582869
6,0,2000-04-03,0.459108


## Using Fugue Transform

Now rewrite the unit test using Fugue transform function. Also we specify the logical partition key.

This step is still unit testable because the input output are both Pandas Dataframes:

In [ ]:
from fugue import transform

transform(generate_data(2), forecast, partition={"by":"unique_id"}, as_local=True)

/databricks/python/lib/python3.8/site-packages/scipy/optimize/_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
 df = fun(x) - f0
/databricks/python/lib/python3.8/site-packages/scipy/optimize/_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
 df = fun(x) - f0
/databricks/python/lib/python3.8/site-packages/scipy/optimize/_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
 df = fun(x) - f0
/databricks/python/lib/python3.8/site-packages/scipy/optimize/_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
 df = fun(x) - f0
/databricks/python/lib/python3.8/site-packages/statsforecast/arima.py:1041: RuntimeWarning: invalid value encountered in sqrt
 return any(np.isnan(np.sqrt(np.diag(obj['var_coef']))))
/databricks/python/lib/python3.8/site-packages/statsforecast/arima.py:1041: RuntimeWarning: invalid value encountered in sqrt
 return any(np.isnan(np.sqrt(np.diag(obj['var_coef']))))
/databricks/python/lib/python3.8/site-packages/scipy/optimize/_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
 df = fun(x) - f0
Out[4]:

,unique_id,ds,auto_arima
0,0,2000-03-28,1.953776
1,0,2000-03-29,1.537709
2,0,2000-03-30,1.210246
3,0,2000-03-31,0.952518
4,0,2000-04-01,0.749675
5,0,2000-04-02,0.590028
6,0,2000-04-03,0.464378
7,1,2000-10-12,0.968096
8,1,2000-10-13,1.253650
9,1,2000-10-14,0.427734


### Test on Spark

We only need to add the parameter `engine` to move the compute to the spark cluster on Databricks (`spark` is the SparkSession).

We also add `num` as a partition parameter to control the balance between overhead and load-balance:

In [ ]:
%%time
transform(generate_data(10000), forecast, partition={"num":500, "by":"unique_id"}, as_local=True, engine=spark)

CPU times: user 3.08 s, sys: 396 ms, total: 3.48 s
Wall time: 2min 10s
Out[6]:

,unique_id,ds,auto_arima
0,303,2000-09-13,2.400590
1,303,2000-09-14,0.504940
2,303,2000-09-15,1.589075
3,303,2000-09-16,0.659683
4,303,2000-09-17,0.927748
...,...,...,...
69995,9796,2000-10-03,1.849620
69996,9796,2000-10-04,1.293470
69997,9796,2000-10-05,0.193491
69998,9796,2000-10-06,0.000000


### Test on Dask

In [ ]:
from dask.distributed import Client
c = Client('127.0.0.1:8786')

In [ ]:
c.scheduler_info()

Out[9]:

Scheduler 
 Scheduler-d9b31a2e-e822-42e8-97b2-572118562d6f 
 
 
 
 Comm: tcp://10.69.232.221:8786
 
 
 Workers: 128
 
 
 
 
 Dashboard: http://10.69.232.221:8787/status 
 
 
 Total threads: 128
 
 
 
 
 Started: 7 minutes ago
 
 
 Total memory: 0.91 TiB
 
 
 
 
 

 
 
 Workers 
 

 
 
 
 
 
 
 Worker: tcp://10.69.226.236:32923 
 
 
 
 
 Comm: tcp://10.69.226.236:32923
 
 
 Total threads: 1
 
 
 
 
 Dashboard: http://10.69.226.236:36655/status 
 
 
 Memory: 7.25 GiB
 
 
 
 
 Nanny: tcp://10.69.226.236:37755
 
 
 
 
 
 Local directory: /dask-worker-space/worker-0n9eoswl
 
 

 

 
 
 
 Tasks executing: 0
 
 
 Tasks in memory: 0
 
 
 
 
 Tasks ready: 0
 
 
 Tasks in flight: 0
 
 
 
 
 CPU usage: 0.0%
 
 
 Last seen: Just now
 
 
 
 
 Memory usage: 91.39 MiB
 
 
 Spilled bytes: 0 B
 
 
 
 
 Read bytes: 1.32 kiB
 
 
 Write bytes: 4.78 kiB
 
 
 

 
 
 
 
 
 
 
 
 
 
 Worker: tcp://10.69.226.236:33207 
 
 
 
 
 Comm: tcp://10.69.226.236:33207
 
 
 Total threads: 1
 
 
 
 
 Dashboard: http://10.69.226.236:37517/status 
 
 
 Memory: 7.25 GiB
 
 
 
 
 Nanny: tcp://10.69.226.236:33441
 
 
 
 
 
 Local directory: /dask-worker-space/worker-8r1a_lny
 
 

 

 
 
 
 Tasks executing: 0
 
 
 Tasks in memory: 0
 
 
 
 
 Tasks ready: 0
 
 
 Tasks in flight: 0
 
 
 
 
 CPU usage: 2.0%
 
 
 Last seen: Just now
 
 
 
 
 Memory usage: 85.14 MiB
 
 
 Spilled bytes: 0 B
 
 
 
 
 Read bytes: 270.0531825849569 B
 
 
 Write bytes: 0.96 kiB
 
 
 

 
 
 
 
 
 
 
 
 
 
 Worker: tcp://10.69.226.236:34337 
 
 
 
 
 Comm: tcp://10.69.226.236:34337
 
 
 Total threads: 1
 
 
 
 
 Dashboard: http://10.69.226.236:33359/status 
 
 
 Memory: 7.25 GiB
 
 
 
 
 Nanny: tcp://10.69.226.236:39261
 
 
 
 
 
 Local directory: /dask-worker-space/worker-vulxc4id
 
 

 

 
 
 
 Tasks executing: 0
 
 
 Tasks in memory: 0
 
 
 
 
 Tasks ready: 0
 
 
 Tasks in flight: 0
 
 
 
 
 CPU usage: 2.0%
 
 
 Last seen: Just now
 
 
 
 
 Memory usage: 91.36 MiB
 
 
 Spilled bytes: 0 B
 
 
 
 
 Read bytes: 0.0 B
 
 
 Write bytes: 0.0 B
 
 
 

 
 
 
 
 
 
 
 
 
 
 Worker: tcp://10.69.226.236:34475 
 
 
 
 
 Comm: tcp://10.69.226.236:34475
 
 
 Total threads: 1
 
 
 
 
 Dashboard: http://10.69.226.236:45309/status 
 
 
 Memory: 7.25 GiB
 
 
 
 
 Nanny: tcp://10.69.226.236:38503
 
 
 
 
 
 Local directory: /dask-worker-space/worker-vq86ryvg
 
 

 

 
 
 
 Tasks executing: 0
 
 
 Tasks in memory: 0
 
 
 
 
 Tasks ready: 0
 
 
 Tasks in flight: 0
 
 
 
 
 CPU usage: 2.0%
 
 
 Last seen: Just now
 
 
 
 
 Memory usage: 86.69 MiB
 
 
 Spilled bytes: 0 B
 
 
 
 
 Read bytes: 6.58 kiB
 
 
 Write bytes: 23.85 kiB
 
 
 

 
 
 
 
 
 
 
 
 
 
 Worker: tcp://10.69.226.236:35189 
 
 
 
 
 Comm: tcp://10.69.226.236:35189
 
 
 Total threads: 1
 
 
 
 
 Dashboard: http://10.69.226.236:41025/status 
 
 
 Memory: 7.25 GiB
 
 
 
 
 Nanny: tcp://10.69.226.236:35041
 
 
 
 
 
 Local directory: /dask-worker-space/worker-wtah66ve
 
 

 

 
 
 
 Tasks executing: 0
 
 
 Tasks in memory: 0
 
 
 
 
 Tasks ready: 0
 
 
 Tasks in flight: 0
 
 
 
 
 CPU usage: 0.0%
 
 
 Last seen: Just now
 
 
 
 
 Memory usage: 91.37 MiB
 
 
 Spilled bytes: 0 B
 
 
 
 
 Read bytes: 1.32 kiB
 
 
 Write bytes: 5.61 kiB
 
 
 

 
 
 
 
 
 
 
 
 
 
 Worker: tcp://10.69.226.236:35431 
 
 
 
 
 Comm: tcp://10.69.226.236:35431
 
 
 Total threads: 1
 
 
 
 
 Dashboard: http://10.69.226.236:43607/status 
 
 
 Memory: 7.25 GiB
 
 
 
 
 Nanny: tcp://10.69.226.236:38733
 
 
 
 
 
 Local directory: /dask-worker-space/worker-ycjluxnt
 
 

 

 
 
 
 Tasks executing: 0
 
 
 Tasks in memory: 0
 
 
 
 
 Tasks ready: 0
 
 
 Tasks in flight: 0
 
 
 
 
 CPU usage: 0.0%
 
 
 Last seen: Just now
 
 
 
 
 Memory usage: 88.83 MiB
 
 
 Spilled bytes: 0 B
 
 
 
 
 Read bytes: 0.0 B
 
 
 Write bytes: 0.0 B
 
 
 

 
 
 
 
 
 
 
 
 
 
 Worker: tcp://10.69.226.236:35667 
 
 
 
 
 Comm: tcp://10.69.226.236:35667
 
 
 Total threads: 1
 
 
 
 
 Dashboard: http://10.69.226.236:37713/status 
 
 
 Memory: 7.25 GiB
 
 
 
 
 Nanny: tcp://10.69.226.236:36039
 
 
 
 
 
 Local dir

The only difference is the value of the `engine` parameter:

In [ ]:
%%time
transform(generate_data(10000), forecast, partition={"num":500, "by":"unique_id"}, as_local=True, engine="dask")

CPU times: user 4.27 s, sys: 405 ms, total: 4.67 s
Wall time: 3min 11s
Out[10]:

,unique_id,ds,auto_arima
0,28,2000-09-22,5.808166
1,28,2000-09-23,2.667787
2,28,2000-09-24,1.198313
3,28,2000-09-25,0.240285
4,28,2000-09-26,0.109615
...,...,...,...
69995,9917,2000-06-14,1.498216
69996,9917,2000-06-15,1.182815
69997,9917,2000-06-16,0.933812
69998,9917,2000-06-17,0.737228
